In [116]:
import plotly.express as px
import seaborn as sns
from datetime import datetime, timedelta, date
from typing import List
import plotly.io as pyo
import math
import pandas as pd
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from datetime import datetime, date, timedelta
from xbbg import blp
import blpapi
import os
import win32com.client as win32
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.io import to_html
from plotly.subplots import make_subplots
import statsmodels.api as sm
from plotly.offline import plot
import scipy.optimize as spop
from scipy.stats import t
from scipy.stats import norm
from statsmodels.regression.rolling import RollingOLS
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.colors as mcolors
import sqlite3
from sql_cache_utils import read_cache, write_cache, append_to_cache, list_cached_tables
from pandas.tseries.offsets import BDay
import scipy.stats as stats

market_configs = {
            'regional': {
                'tickers': ['MXWO Index', 'SPX Index', 'SPW Index', 'NKY Index', 'AS51 Index', 'AS38 Index', 
                            'MXEF Index', 'SXXP Index', 'MXKR Index', 'MXCN Index', 'UKX Index', 'MXWOU Index', 'NIFTY Index'],
                'names': ['World', 'US', 'US Equal Weight', 'Japan', 'Australia', 'Aussie Smalls', 
                          'EM', 'Europe', 'Korea', 'China', 'UK', 'World ex US', 'India'],
                'metric_fields': ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 
                                  'CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO'],
                'metric_names': ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales'],
                'cache_prefix': 'regional',
                'earnings_tickers': ['MXWO Index', 'SPX Index', 'SPW Index', 'NKY Index', 'AS51 Index', 
                                    'MXEF Index', 'SXXP Index', 'MXKR Index', 'MXCN Index', 'UKX Index','MXWOU Index', 'NIFTY Index'],
                'earnings_names': ['World', 'US', 'US_Equal_Weight', 'Japan', 'Australia', 
                                  'Emerging Markets', 'Europe', 'Korea', 'China', 'UK', 'World ex US', 'India']
            },
            'factor': {
                'tickers': ['MXWO000V Index', 'M1WOEV Index', 'MXWO000G Index', 'M1WOQU Index', 
                           'MXWOSC Index', 'MXWOLC Index', 'M1WOMVOL Index'],
                'names': ['Value', 'Enhanced Value', 'Growth', 'Quality', 'Small', 'Large', 'Low Volatility'],
                'metric_fields': ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 
                                  'CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO'],
                'metric_names': ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales'],
                'cache_prefix': 'factor',
                'earnings_tickers': ['MXWO000V Index','M1WOEV Index', 'MXWO000G Index', 'M1WOQU Index', 
                                    'MXWOSC Index', 'MXWOLC Index', 'M1WOMVOL Index'],
                'earnings_names': ['Value','Enhanced Value', 'Growth', 'Quality', 'Small', 'Large', 'Low_Volatility']
            },
            'aussie': {
                'tickers': ['AS51BANX Index', 'AS45 Index', 'MVMVWTRG Index', 'AS51MATL Index', 'AS51 Index'],
                'names': ['Banks', 'Resources', 'Equal Weight', 'Materials', 'ASX 200'],
                'metric_fields': ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 'PX_TO_SALES_RATIO'],
                'metric_names': ['Forward PE', 'Price to Book', 'CAPE', 'Price to Sales'],
                'cache_prefix': 'aussie',
                'earnings_tickers': ['AS51BANX Index', 'AS45 Index', 'MVMVWTRG Index', 'AS51MATL Index', 'AS52 Index'],
                'earnings_names': ['Banks', 'Resources', 'Equal Weight', 'Materials', 'ASX 200']
            },
            'sector': {
                'regions': {
                    'us': {
                        'tickers': ['s5matr Index', 's5enrs index', 's5finl index', 's5hlth index', 's5cond index', 
                                   's5cons index', 's5inft index', 's5rlst index', 's5util index', 's5tels index', 's5indu index'],
                        'cache_prefix': 'us_sector'
                    },
                    'au': {
                        'tickers': ['AS51MATL Index', 'as51engy index', 'as51fin index', 'as51hc index', 'as51cond index', 
                                   'as51cons index', 'as51it index', 'as51prop index', 'as51util index', 'as51tele index', 'as51indu index'],
                        'cache_prefix': 'au_sector'
                    },
                    'jp': {
                        'tickers': ['MXJP0MT Index', 'MXJP0EN Index', 'MXJP0FN Index', 'MXJP0HC Index', 'MXJP0CD Index', 
                                   'MXJP0CS Index', 'MXJP0IT Index', 'MXJP0RL Index', 'MXJP0UT Index', 'MXJP0TC Index', 'MXJP0IN Index'],
                        'cache_prefix': 'jp_sector'
                    },
                    'uk': {
                        'tickers': ['MXGB0MT Index', 'MXGB0EN Index', 'MXGB0FN Index', 'MXGB0HC Index', 'MXGB0CD Index', 
                                   'MXGB0CS Index', 'MXGB0IT Index', 'MXGB0RL Index', 'MXGB0UT Index', 'MXGB0TC Index', 'MXGB0IN Index'],
                        'cache_prefix': 'uk_sector'
                    },
                    'eu': {
                        'tickers': ['MXEU0MT Index', 'MXEU0EN Index', 'MXEU0FN Index', 'MXEU0HC Index', 'MXEU0CD Index', 
                                   'MXEU0CS Index', 'MXEU0IT Index', 'MXEU0RE Index', 'MXEU0UT Index', 'MXEU0TC Index', 'MXEU0IN Index'],
                        'cache_prefix': 'eu_sector'
                    }
                },
                'names': ['Material', 'Energy', 'Financials', 'Healthcare', 'Consumer Discret', 'Consumer Staples', 
                         'Info Tech', 'Real Estate', 'Utilities', 'Communication Serv', 'Industrials'],
                'metric_fields': ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 
                                 'CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO'],
                'metric_names': ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales']
            }
        }

In [1]:
def valuation_pdf(ticker_insert):
    # Shared Constants
    FONT = "Montserrat"
    START_DATE = '1997-05-10'
    AUS_START_DATE = '2003-05-10'
    LOWER = 0.01
    UPPER = 0.99
    returns = 'TOT_RETURN_INDEX_GROSS_DVDS'
    PER = 'M'

    names_of_vals = [
        'Price to Book',
        'CAPE',
        'EV/Trailing EBITDA',
        'Price to Sales'
    ]

    valuations = [
        'PX_TO_BOOK_RATIO',
        'LONG_TERM_PRICE_EARNINGS_RATIO',
        'CURRENT_EV_TO_T12M_EBITDA',
        'PX_TO_SALES_RATIO'
    ]

    bloomberg_ticker = ticker_insert
    bloomberg_name = blp.bdp(bloomberg_ticker, 'long_comp_name')
    country_tag = bloomberg_name['long_comp_name'][0]
    
    forward_pe_multi_chart = [ticker_insert,'MXWO Index', 'MXEF Index']
    ticker_name = blp.bdp(ticker_insert, 'long_comp_name')['long_comp_name'][0]

    using_vs_dm_and_em = [ticker_name,'Developed Mkt Equities','Emerging Mkt Equities']

    ############ REG PRICE #############    
    regular_price_action = blp.bdh(forward_pe_multi_chart, flds = returns, start_date=START_DATE, Per=PER, FX="AUD").droplevel(axis=1,level=1)
    regular_price_action.index = pd.to_datetime(regular_price_action.index)
    regular_price_action.columns = using_vs_dm_and_em

    latest_date = regular_price_action.iloc[-1] if regular_price_action.iloc[-1].notna().any() else regular_price_action.iloc[-2]
    ret3 = ((regular_price_action.iloc[-1] / regular_price_action.iloc[-4])-1)
    ret6 = ((regular_price_action.iloc[-1] / regular_price_action.iloc[-7])-1)
    ret1y = ((regular_price_action.iloc[-1] / regular_price_action.iloc[-13])-1)
    ret3y = ((regular_price_action.iloc[-1] / regular_price_action.iloc[-37])**(1/3)-1)
    ret5y = ((regular_price_action.iloc[-1] / regular_price_action.iloc[-61])**(1/5)-1)
    ret10y = ((regular_price_action.iloc[-1] / regular_price_action.iloc[-121])**(1/10)-1)
    ret20y = ((regular_price_action.iloc[-1] / regular_price_action.iloc[-241])**(1/20)-1)
    
    returns_df = pd.DataFrame({
        "3M Return": ret3 * 100,
        "6M Return": ret6 * 100,
        "1Y Return": ret1y * 100,
        "3Y Annualized": ret3y * 100,
        "5Y Annualized": ret5y * 100,
        "10Y Annualized": ret10y * 100,
        "20Y Annualized": ret20y * 100
    }).round(2).astype(str) + '%'

    since_1999 = (1+regular_price_action.pct_change()).cumprod().dropna()
    gfc = (1+regular_price_action.loc['2007-12-31':'2011-03-31'].pct_change()).cumprod().dropna()
    since_covid = (1+regular_price_action.loc['2020-03-31':].pct_change()).cumprod().dropna()
    vol = (regular_price_action.pct_change().rolling(window=24).std()*math.sqrt(12)).dropna()
    mean = (regular_price_action.pct_change().rolling(window=24).mean()*(12)).dropna()
    return_over_risk = mean / vol

    def winsorize_df(df, limits=(0.01, 0.01)):
                df_winsorized = df.copy()
                for col in df.columns:
                    if pd.api.types.is_numeric_dtype(df[col]):
                        non_nan = df[col].dropna()
                        winsorized = stats.mstats.winsorize(non_nan, limits=limits)
                        df_winsorized.loc[non_nan.index, col] = winsorized
                return df_winsorized

    bookvalue = blp.bdh(forward_pe_multi_chart, flds = ['TANG_BOOK_VAL_PER_SH','BOOK_VAL_PER_SH'], start_date=START_DATE, Per=PER) #.droplevel(axis=1,level=1)
    ratio_assets = bookvalue.xs('TANG_BOOK_VAL_PER_SH', axis =1, level =1) / bookvalue.xs('BOOK_VAL_PER_SH', axis =1, level =1)
    bookvalue_win = winsorize_df(ratio_assets)
    bookvalue_win = bookvalue_win.rolling(window=6).median()*100
    bookvalue_win.columns = using_vs_dm_and_em

    chart_99 = px.line(
                since_1999,
                title="Total Return (Dividends Reinvested)",
                color_discrete_sequence=["#30415F",  "#DBDBDB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
            )

    chart_99.update_layout(
        width=850,
        height=500,
        title=dict(
            text=f'<b><span style="color:black;">Total Return (Dividends Reinvested)</span></b>',
            font=dict(size=18)
        ),
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5,
            font=dict(size=11)
        )
    )

    chart_99.update_xaxes(
        tickangle=45,
        tickfont=dict(size=10),
        title_text=""
    )

    chart_99.update_yaxes(
        #range=[0, 20],     
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False
    )

    gfc_chart = px.line(
                gfc,
                title="Total Return during GFC (Dividends Reinvested)",
                color_discrete_sequence=["#30415F",  "#DBDBDB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
            )

    gfc_chart.update_layout(
        width=850,
        height=500,
        title=dict(
            text=f'<b><span style="color:black;">Total Return during GFC (Dividends Reinvested)</span></b>',
            font=dict(size=18)
        ),
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5,
            font=dict(size=11)
        )
    )

    gfc_chart.update_xaxes(
        tickangle=45,
        tickfont=dict(size=10),
        title_text=""
    )

    gfc_chart.update_yaxes(
        #range=[0, 20],     
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False
    )

    since_covid_chart = px.line(
                since_covid,
                title="Total Return since COVID (Dividends Reinvested)",
                color_discrete_sequence=["#30415F",  "#DBDBDB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
            )

    since_covid_chart.update_layout(
        width=850,
        height=500,
        title=dict(
            text=f'<b><span style="color:black;">Total Return since COVID (Dividends Reinvested)</span></b>',
            font=dict(size=18)
        ),
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5,
            font=dict(size=11)
        )
    )

    since_covid_chart.update_xaxes(
        tickangle=45,
        tickfont=dict(size=10),
        title_text=""
    )

    since_covid_chart.update_yaxes(
        #range=[0, 20],     
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False
    )

    rolling_3y_vol_chart = px.line(
                vol,
                title="Rolling Volatility (2Y)",
                color_discrete_sequence=["#30415F",  "#DBDBDB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
            )

    rolling_3y_vol_chart.update_layout(
        width=850,
        height=500,
        title=dict(
            text=f'<b><span style="color:black;">Rolling Volatility (2Y)</span></b>',
            font=dict(size=18)
        ),
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5,
            font=dict(size=11)
        )
    )

    rolling_3y_vol_chart.update_xaxes(
        tickangle=45,
        tickfont=dict(size=10),
        title_text=""
    )

    rolling_3y_vol_chart.update_yaxes(
        #range=[0, 20],     
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False
    )

    return_over_risk_chart = px.line(
                return_over_risk,
                title="Rolling Return / Risk (Risk-adjusted Return) (2Y)",
                color_discrete_sequence=["#30415F",  "#DBDBDB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
            )

    return_over_risk_chart.update_layout(
        width=850,
        height=500,
        title=dict(
            text=f'<b><span style="color:black;">Rolling Return / Risk (Risk-adjusted Return) (2Y)</span></b>',
            font=dict(size=18)
        ),
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5,
            font=dict(size=11)
        )
    )

    return_over_risk_chart.update_xaxes(
        tickangle=45,
        tickfont=dict(size=10),
        title_text=""
    )

    return_over_risk_chart.update_yaxes(
        #range=[0, 20],     
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False
    )

    bookvalue_win_chart = px.line(
                bookvalue_win,
                title="Assets that are Tangible",
                color_discrete_sequence=["#30415F",  "#DBDBDB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
            )

    bookvalue_win_chart.update_layout(
        width=850,
        height=500,
        title=dict(
            text=f'<b><span style="color:black;">% of Assets that are Tangible</span></b>',
            font=dict(size=18)
        ),
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5,
            font=dict(size=11)
        )
    )

    bookvalue_win_chart.update_xaxes(
        tickangle=45,
        tickfont=dict(size=10),
        title_text=""
    )

    bookvalue_win_chart.update_yaxes(
        #range=[0, 20],     
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False
    )

    ############ TIME SERIES VALUATION #############

    def time_series_valuations_and_table_of_traffic_lights(bloomberg_ticker, forward_pe_multi_chart, valuations, START_DATE, PER, ticker_name):
        data = blp.bdh(bloomberg_ticker, flds = valuations, start_date=START_DATE, Per=PER).droplevel(axis=1,level=0)
        benchmark_data_fwd_pe = blp.bdh(forward_pe_multi_chart, flds = 'BEST_PE_RATIO', start_date=START_DATE, Per=PER).droplevel(axis=1,level=1)
        
        def winsorize_df(df, limits=(0.005, 0.005)):
            df_winsorized = df.copy()
            for col in df.columns:
                if pd.api.types.is_numeric_dtype(df[col]):
                    non_nan = df[col].dropna()
                    winsorized = stats.mstats.winsorize(non_nan, limits=limits)
                    df_winsorized.loc[non_nan.index, col] = winsorized
            return df_winsorized

        benchmark_data_fwd_pe = winsorize_df(benchmark_data_fwd_pe)
        benchmark_data_fwd_pe.columns = [ticker_name,'Developed Mkt Equities','Emerging Mkt Equities']

        xs = pd.DataFrame(index=benchmark_data_fwd_pe.index)
        xs[f'{ticker_name} / DM'] = benchmark_data_fwd_pe[ticker_name] / benchmark_data_fwd_pe['Developed Mkt Equities']
        xs[f'{ticker_name} / EM'] = benchmark_data_fwd_pe[ticker_name] / benchmark_data_fwd_pe['Emerging Mkt Equities']

        data_win = winsorize_df(data)
        data_win.columns = names_of_vals
        z_score = (data_win - data_win.mean()) / data_win.std()
        z_score = z_score.ffill()
        z_score['Composite Valuation'] = z_score.mean(axis=1)
        data_win['Composite Valuation (CAPE, PS, PB, EV/EBITDA)'] = z_score['Composite Valuation']

        df = data_win.round(3) ### THE OFFICIAL CLEANED DF !!!!!!!!!!!!!!!!

        fwd_pe_chart = px.line(
            benchmark_data_fwd_pe,
            title="Forward PE (Blended 1Y Fwd Earnings) vs Benchmarks",
            color_discrete_sequence=["#30415F",  "#DBDBDB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
        )

        fwd_pe_chart.update_layout(
            width=850,
            height=500,
            title=dict(
                text=f'<b><span style="color:black;">Forward PE (Blended 1Y Fwd Earnings) vs Benchmarks</span></b>',
                font=dict(size=16)
            ),
            font=dict(family="Montserrat", size=13),
            plot_bgcolor='white',
            showlegend=True,
            legend=dict(
                orientation='h',
                yanchor='bottom',
                y=-0.3,
                xanchor='center',
                x=0.5,
                font=dict(size=11)
            )
        )

        fwd_pe_chart.update_xaxes(
            tickangle=45,
            tickfont=dict(size=10),
            title_text=""
        )

        fwd_pe_chart.update_yaxes(
            range=[0, 30],     
            showgrid=True,
            gridcolor='lightgray',
            zeroline=False
        )

        # --- # --- # --- XS PLOT
        xs = xs.dropna()    
        xs_chart = px.line(
            xs,
            title="Relative Valuations (To DM & EM Benchmarks)",
            color_discrete_sequence=["#30415F",  "#B8ABAB", "#82ADCF", "#2E8B57", "#CF9660", "#7CE1F3"]
        )

        xs_chart.update_layout(
            width=850,
            height=500,
            title=dict(
                text=f'<b><span style="color:black;">Relative Valuations (To DM & EM Benchmarks)</span></b>',
                font=dict(size=16)
            ),
            font=dict(family="Montserrat", size=13),
            plot_bgcolor='white',
            showlegend=True,
            legend=dict(
                orientation='h',
                yanchor='bottom',
                y=-0.3,
                xanchor='center',
                x=0.5,
                font=dict(size=11)
            )
        )

        xs_chart.update_xaxes(
            tickangle=45,
            tickfont=dict(size=10),
            title_text=""
        )

        xs_chart.update_yaxes(
           # range=[0, 30],     
            showgrid=True,
            gridcolor='lightgray',
            zeroline=False
        )

        df_dict = {}
        for col in df.columns:
            series = df[col]
            median = series.median()
            std = series.std()
            df_col = pd.DataFrame({
                col: series,
                'median': median,
                '+1_std': median + std,
                '-1_std': median - std,
                '+2_std': median + 2*std,
                '-2_std': median - 2*std
            }, index=series.index)
            df_dict[col] = df_col  # Store in dictionary with key as column name


        from plotly.subplots import make_subplots
        import plotly.graph_objects as go
        from math import ceil

        # --- Sort and split Composite from others ---
        # Manually reorder keys to move "Composite Valuation" first
        custom_order = ['Composite Valuation (CAPE, PS, PB, EV/EBITDA)', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales']
        ordered_dict = {k: df_dict[k] for k in custom_order if k in df_dict}

        composite_key = 'Composite Valuation (CAPE, PS, PB, EV/EBITDA)'
        other_keys = [k for k in custom_order if k in df_dict and k != composite_key]

        # Final count of rows: 1 for Composite + ceil(len(others) / 2)
        num_sub_rows = 1 + ceil(len(other_keys) / 2)

        # Create subplot grid with Composite spanning first row
        fig = make_subplots(
            rows=num_sub_rows, cols=2,
            specs=[[{"colspan": 2}, None]] + [[{}, {}] for _ in range(num_sub_rows - 1)],
            subplot_titles=[f"<b>{composite_key}</b>"] + [f"<b>{k}</b>" for k in other_keys],
            vertical_spacing=0.15,
            horizontal_spacing=0.05,
            shared_xaxes=False,
            shared_yaxes=False
        )

        # --- Add Composite Valuation chart to row 1, col 1 with colspan ---
        data = df_dict[composite_key]
        fig.add_trace(go.Scatter(
            x=data.index, y=data[composite_key], mode='lines',
            line=dict(color="#4682B4", width=2.5),
            name=composite_key, showlegend=False
        ), row=1, col=1)

        # Statistical bands for Composite
        for band in ['median', '+1_std', '-1_std', '+2_std', '-2_std']:
            fig.add_trace(go.Scatter(
                x=data.index, y=data[band], mode='lines',
                line=dict(color='lightgray', dash='dot' if '+' in band or '-' in band else 'solid'),
                showlegend=False
            ), row=1, col=1)

        fig.update_layout(
            width=1250,
            height=300 * num_sub_rows,
            title=dict(
                text=f'<b><span style="color:black;">{country_tag} - Valuation</span></b>',
                font=dict(size=18)
            ),
            font=dict(family="Montserrat", size=13),
            plot_bgcolor='white',
            showlegend=False,
            xaxis=dict(domain=[0.16, 0.84]),  # Correct way to narrow Composite's width
        )

        # --- Add remaining charts starting from row 2 ---
        for i, key in enumerate(other_keys):
            data = df_dict[key]
            row = 2 + i // 2
            col = 1 + i % 2
            main_color = '#30415f'

            fig.add_trace(go.Scatter(
                x=data.index, y=data[key], mode='lines',
                line=dict(color=main_color, width=2.5),
                name=key, showlegend=False
            ), row=row, col=col)

            for band in ['median', '+1_std', '-1_std', '+2_std', '-2_std']:
                fig.add_trace(go.Scatter(
                    x=data.index, y=data[band], mode='lines',
                    line=dict(color='lightgray', dash='dot' if '+' in band or '-' in band else 'solid'),
                    showlegend=False
                ), row=row, col=col)


        # Layout settings
        fig.update_layout(
            width=1250,
            height=300 * num_sub_rows,
            title=dict(
                text=f'<b><span style="color:black;">{country_tag} - Valuation</span></b>',
                font=dict(size=18)
            ),
            font=dict(family="Montserrat", size=13),
            plot_bgcolor='white',
            showlegend=False
        )

        fig.update_xaxes(tickangle=45, tickfont=dict(size=10), title_text="")\
        
        return xs_chart, fwd_pe_chart, fig, df

    xs_chart, fwd_pe_chart , ts_val_charts, data = time_series_valuations_and_table_of_traffic_lights(bloomberg_ticker, forward_pe_multi_chart, valuations, START_DATE, PER, ticker_name)


    ############ EARNINGS #############

    def earnings(ticker_wanted):
        from dateutil.relativedelta import relativedelta
        last_yr = (datetime.today().date() - relativedelta(years=1)).strftime('%Y-%m-%d')

        FONT = "Montserrat"
        START_DATE_E = last_yr
        LOWER = 0.01
        UPPER = 0.99

        fwd = blp.bdh(tickers = [ticker_wanted, 'MXWO Index', 'MXEF Index'], flds = ['INDX_WEIGHTED_EST_ERN'], start_date = START_DATE_E,BEST_FPERIOD_OVERRIDE='BF') #.droplevel(axis=1,level=0)
        t12 = blp.bdh(tickers = [ticker_wanted, 'MXWO Index', 'MXEF Index'], flds = ['t12_eps_aggte'], start_date = START_DATE_E) #.droplevel(axis=1,level=0)
        forward_eps_impulse = blp.bdh(tickers = ticker_wanted, flds = ['INDX_WEIGHTED_EST_ERN'], start_date = '1997-01-01', Per='M', BEST_FPERIOD_OVERRIDE='BF').droplevel(axis=1,level=0)

        fwd.index = pd.to_datetime(fwd.index)
        t12.index = pd.to_datetime(t12.index)
        forward_eps_impulse.index = pd.to_datetime(forward_eps_impulse.index)

        earnings_revisions = (fwd.xs('INDX_WEIGHTED_EST_ERN',axis=1,level=1) / t12.xs('t12_eps_aggte',axis=1,level=1)) - 1
        # earnings_revisions.columns = using_vs_dm_and_em
        earnings_impulse = forward_eps_impulse.pct_change(12).dropna()
        earnings_impulse.columns = ['Earnings Forecast YoY (Blended 12M Forward)']
        earnings_impulse[0] = 0


        # --- # ---- # --- # ---- # --- # ---- # --- # ---- # --- # ---- # --- # ---- # --- # ---- # --- # ----

        earnings_revisions_chart = px.line(
            earnings_revisions,
            title="Earnings Revision % (Blended Forward 12 Months)",
            color_discrete_sequence=["#30415F", "#A4B6C5", "#2E8B57", "#DBDBDB", "#CF9660", "#7CE1F3"]
        )

        earnings_revisions_chart.update_layout(
            width=800,
            height=450,
            title=dict(
                text=f'<b><span style="color:black;">Earnings Revision % (Blended Forward 12 Months)</b>', #text=f'<b><span style="color:black;">Earnings Revision % (Blended Forward 12 Months): {country_tag}</b>',
                font=dict(size=18)
            ),
            font=dict(family="Montserrat", size=13),
            plot_bgcolor='white',
            showlegend=True,
            legend=dict(
                orientation='h',
                yanchor='bottom',
                y=-0.3,
                xanchor='center',
                x=0.5,
                font=dict(size=11)
            )
        )

        earnings_revisions_chart.update_xaxes(
            tickangle=45, 
            tickfont=dict(size=10),
            title_text=""
        )

        earnings_revisions_chart.update_yaxes(
        # range=[5, 30],
            showgrid=True, 
            gridcolor='lightgray',
            zeroline=False, title_text = "Earnings Revisions YoY", tickformat=".0%"  # format ticks as whole percentages
        )

        # --- # ----

        earnings_impulse_chart = px.line(
            earnings_impulse,
            title="Forward PE (Blended 1Y Fwd Earnings) vs Benchmarks",
            color_discrete_sequence=["#30415F", "#A4B6C5", "#2E8B57", "#DBDBDB", "#CF9660", "#7CE1F3"]
        )

        earnings_impulse_chart.update_layout(
            width=800,
            height=400,
            title=dict(
                text=f'<b><span style="color:black;">Earnings Impulse:</b>', #text=f'<b><span style="color:black;">Earnings Impulse: {country_tag}</b>',
                font=dict(size=18)
            ),
            font=dict(family="Montserrat", size=13),
            plot_bgcolor='white',
            showlegend=True,
            legend=dict(
                orientation='h',
                yanchor='bottom',
                y=-0.3,
                xanchor='center',
                x=0.5,
                font=dict(size=11)
            )
        )

        earnings_impulse_chart.update_xaxes(
            tickangle=45,
            tickfont=dict(size=10),
            title_text=""
        )

        earnings_impulse_chart.update_yaxes(
        # range=[5, 30],
            showgrid=True,
            gridcolor='lightgray',
            zeroline=False, title_text = "Earnings Forecast YoY", tickformat=".0%"  # format ticks as whole percentages
        )

        return earnings_impulse_chart, earnings_revisions_chart

    earnings_imp_chart, earnings_rev_chart = earnings(ticker_insert)



    ############ MAKING THE TABLE FOR THE VALUATIONS #############

    from scipy.stats import percentileofscore
    latest = data.iloc[-1]
    one_ago = data.iloc[-13]
    latest_percentiles = {
        col: percentileofscore(data[col].dropna(), latest[col], kind='mean')
        for col in data.columns
    }
    one_ago_percentiles = {
        col: percentileofscore(data[col].dropna(), one_ago[col], kind='mean')
        for col in data.columns
    }
    percentile_df = pd.DataFrame([latest.to_dict(), latest_percentiles, one_ago_percentiles], index=[f'Value at {data.index[-1]}', 'Current Percentile', '1Y Ago Percentile']).round(2)
    #percentile_df['1Y Percentile Change'] = percentile_df[f'{df.index[-1]}'] - percentile_df['1Y Ago']

    def classify_percentile(p):
        try:
            p = float(p)
        except (ValueError, TypeError):
            return ''  # or return p if you want to keep the original

        if p >= 80:
            return "Very Expensive"
        elif p >= 60:
            return "Expensive"
        elif p >= 40:
            return "Neutral"
        elif p >= 20:
            return "Cheap"
        else:
            return "Very Cheap"

    # Apply classification to numeric values only
    latest_row = percentile_df.loc['Current Percentile']
    classification_row = latest_row.apply(classify_percentile)

    # Add it as a new row
    percentile_df.loc["Valuation Classification"] = classification_row
    per_T = percentile_df.T

    return return_over_risk_chart, returns_df, chart_99, gfc_chart, since_covid_chart, rolling_3y_vol_chart, bookvalue_win_chart, xs_chart, fwd_pe_chart, ts_val_charts, earnings_imp_chart, earnings_rev_chart, per_T, data

In [2]:
def style_html_table(df, title):
    # Color mapping
    colors = {
        "very expensive": "#D9534F",
        "expensive": "#F0AD4E", 
        "neutral": "#FFC107",
        "cheap": "#5CB85C",
        "very cheap": "#4CAF50"
    }
    
    html = f"""
    <h3 style="font-family: Montserrat, sans-serif; font-weight: 700; font-size: 18px; color: #30415f; margin-top: 30px;">
        {title}
    </h3>
    <div style="
        background-color: white;
        border-radius: 8px;
        box-shadow: 0 2px 6px rgba(0,0,0,0.05);
        padding: 10px;
        margin-bottom: 20px;
        overflow-x: auto;
        font-family: Montserrat, sans-serif;
        font-size: 13px;
        max-width: fit-content;
        margin-left: auto;
        margin-right: auto;
    ">
    <table style="border-collapse: collapse; width: auto; min-width: 400px;">
        <thead>
            <tr>
                <th style="text-align: left; padding: 10px 8px; background-color: #30415f; color: white; white-space: nowrap;">Metric</th>
    """
    
    # Add column headers
    for col in df.columns:
        html += f'<th style="text-align: center; padding: 10px 8px; background-color: #30415f; color: white; white-space: nowrap;">{col}</th>'
    
    html += """
            </tr>
        </thead>
        <tbody>
    """
    
    # Add data rows
    for idx, row in df.iterrows():
        html += f"<tr><td style='padding: 10px 8px; font-weight: bold; color: black; white-space: nowrap; background-color: #f8f9fa;'>{idx}</td>"
        
        for col in df.columns:
            val = row[col]
            
            # Default cell style
            cell_style = "padding: 1px 8px; text-align: center; white-space: nowrap;"
            
             # Check if this is the composite valuation row (make all cells light gray except valuation classification)
            idx_lower = str(idx).strip().lower()
            if "composite valuation" in idx_lower:
                if col != "Valuation Classification":
                    cell_style += " background-color: #f8f9fa;"
            
            # Check if this is the Valuation Classification column
            if col == "Valuation Classification":
                val_clean = str(val).strip().lower()
                if val_clean in colors:
                    bg_color = colors[val_clean]
                    cell_style += f" background-color: {bg_color}; color: white; font-weight: bold;"
            
            # Format the value
            if isinstance(val, float):
                formatted_val = f"{val:.2f}"
            else:
                formatted_val = str(val)
            
            html += f"<td style='{cell_style}'>{formatted_val}</td>"
        
        html += "</tr>"
    
    html += """
        </tbody>
    </table>
    </div>
    """
    return html

## INSERT THE REGION/FACTOR/AREA U WANT:

In [ ]:
ticker_wanted = 'MXIN Index'
country_tag = 'India'

In [ ]:
# DECOMPOSED #
for_ytd_equity_source_attrib = [ticker_wanted,'SPX Index', 'MXCN Index','AS51 Index','UKX Index','MXEU Index']
names_for_ytd_equity_source_attrib = [country_tag,'US','China','Australia','UK','EU']
FONT = "Montserrat"
START_DATE = '2020-05-10'
START_DATE_E = '2024-12-31'
returns = ['px_last','TOT_RETURN_INDEX_GROSS_DVDS'] 
e_fwd_raw = blp.bdh(for_ytd_equity_source_attrib, flds = 'INDX_WEIGHTED_EST_ERN', start_date=START_DATE_E,BEST_FPERIOD_OVERRIDE='BF').droplevel(axis=1,level=1)
e_backward_raw = blp.bdh(for_ytd_equity_source_attrib, flds = 'T12_EPS_AGGTE', start_date=START_DATE_E).droplevel(axis=1,level=1)
eps_12 = (e_fwd_raw / e_backward_raw) - 1
eps_12.index = pd.to_datetime(eps_12.index)

raw_price_return_indices = blp.bdh(for_ytd_equity_source_attrib, flds = returns, start_date=START_DATE) #.droplevel(axis=1,level=1)
raw_price_return_indices.index = pd.to_datetime(raw_price_return_indices.index)

price_index = raw_price_return_indices.xs('px_last', axis=1,level=1)
total_return = raw_price_return_indices.xs('TOT_RETURN_INDEX_GROSS_DVDS', axis=1,level=1)

# EARNING
first_date_this_year = eps_12.index[eps_12.index.year == pd.Timestamp.today().year].min() # Start of Year
mask = eps_12.notnull().all(axis=1)
latest_complete_row = eps_12[mask].iloc[-1] # Last date with all.
earnings_contrib = latest_complete_row - eps_12.loc[first_date_this_year] # earnings

# DIVIDEND
ytd_total_return = (total_return.dropna().iloc[-1] / total_return.loc[total_return.index[total_return.index.year == pd.Timestamp.today().year - 1].max()]) - 1
ytd_price = (price_index.dropna().iloc[-1] / price_index.loc[price_index.index[price_index.index.year == pd.Timestamp.today().year - 1].max()]) - 1
dividend_contrib = ytd_total_return - ytd_price

# VALUATION
valuation_contrib = ytd_price - earnings_contrib

# COMBINE
equity_sources = pd.concat([dividend_contrib, earnings_contrib, valuation_contrib], axis = 1)
equity_sources['Total Local Return'] = equity_sources.sum(axis=1)
equity_sources.columns = ['Dividend','Earnings','Valuation','Total Local Return']
equity_sources = (equity_sources*100).round(2)
equity_sources.index = names_for_ytd_equity_source_attrib


## EARNINGS DECOMP
import plotly.graph_objects as go
import pandas as pd

# Split into positive and negative components
positive = equity_sources.clip(lower=0)
negative = equity_sources.clip(upper=0)

# Total return = full sum of all components
total_return = equity_sources.sum(axis=1)

fig = go.Figure()

# Add NEGATIVE components first (hidden from legend)
for col, color in zip(['Earnings', 'Dividend', 'Valuation'], ['#e0e0e0', "#89c0e4", "#30415f"]):
    fig.add_trace(go.Bar(
        name=f'{col} (Neg)',
        x=equity_sources.index,
        y=negative[col],
        marker_color=color,
        showlegend=False
    ))

# Add POSITIVE components
for col, color in zip(['Earnings', 'Dividend', 'Valuation'], ["#e0e0e0", '#89c0e4', '#30415f']):
    fig.add_trace(go.Bar(
        name=col,
        x=equity_sources.index,
        y=positive[col],
        marker_color=color
    ))

# Add Total Return dots
fig.add_trace(go.Scatter(
    name='Total Return',
    x=equity_sources.index,
    y=total_return,
    mode='markers',
    marker=dict(color='orange', size=10),
    showlegend=True
))

# Add zero line
fig.add_shape(
    type="line",
    x0=-0.5,
    x1=len(equity_sources) - 0.5,
    y0=0,
    y1=0,
    line=dict(color="black", width=1),
    xref="x",
    yref="y"
)

# Layout styling
fig.update_layout(
    width=850,
    barmode='relative',
    title=dict(
    text='<b>Equity Sources of Total Return – Year to Date</b>',
    font=dict(size=20, family='Montserrat, sans-serif')
),

    title_font=dict(size=20, family='Montserrat, sans-serif'),
    font=dict(family='Montserrat, sans-serif', size=12),
    yaxis_title='Total Return (%)',
    yaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.05)'),
    xaxis=dict(showgrid=False),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=40, r=40, t=60, b=100),  # More bottom space for legend
    legend=dict(
        orientation='h',
        yanchor='top',
        y=-0.25,     # Push legend below plot
        xanchor='center',
        x=0.5,
        font=dict(size=11)
    )
)

In [ ]:
# --- Main Code
name_for_report = blp.bdp(ticker_wanted,'long_comp_name')['long_comp_name'][0]
return_over_risk_chart, returns_df, chart_99, gfc_chart, since_covid_chart, rolling_3y_vol_chart, bookvalue_win_chart, xs_chart, fwd_pe_chart, ts_val_charts, earnings_imp_chart, earnings_rev_chart, per_T, data = valuation_pdf(ticker_wanted)

# --- HTMl'ING
# --- Deocomp
decomp_html = fig.to_html(include_plotlyjs=False, full_html=False)

# --- Price Section
master_return_table = style_html_table(returns_df,"")
chart_99_html = chart_99.to_html(include_plotlyjs=False, full_html=False)
gfc_chart_html = gfc_chart.to_html(include_plotlyjs=False, full_html=False)
since_covid_chart_html = since_covid_chart.to_html(include_plotlyjs=False, full_html=False)
rolling_3y_vol_chart_html = rolling_3y_vol_chart.to_html(include_plotlyjs=False, full_html=False)
bookvalue_win_chart_html = bookvalue_win_chart.to_html(include_plotlyjs=False, full_html=False)
return_over_risk_chart_html = return_over_risk_chart.to_html(include_plotlyjs=False, full_html=False)

# --- Valuations Section
table_traffic = style_html_table(per_T, "")
fwd_pe_chart_html = fwd_pe_chart.to_html(include_plotlyjs=False, full_html=False)
ts_val_charts_html = ts_val_charts.to_html(include_plotlyjs=False, full_html=False)
xs_chart_html = xs_chart.to_html(include_plotlyjs=False, full_html=False)

# --- Earnings Section
earnings_imp_chart_html = earnings_imp_chart.to_html(include_plotlyjs=False, full_html=False)
earnings_rev_chart_html = earnings_rev_chart.to_html(include_plotlyjs=False, full_html=False)

Execute:

In [136]:
import plotly.graph_objects as go
from jinja2 import Template
from datetime import datetime

# HTML template
template_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>{{ report_title }}</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/plotly.js/2.26.0/plotly.min.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Montserrat&display=swap" rel="stylesheet">
    <style>
        body { font-family: 'Montserrat', sans-serif; margin: 0; padding: 20px; background-color: #f8f9fa; color: #333; line-height: 1.6; }
        .container { max-width: 1200px; margin: 0 auto; background: white; padding: 40px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
        .header { text-align: center; margin-bottom: 40px; border-bottom: 2px solid #30415f; padding-bottom: 20px; }
        .header h1 { color: #30415f; margin: 0; font-size: 2.5em; }
        .header .date { color: #666; font-size: 1.1em; margin-top: 10px; }
        .section { margin-bottom: 40px; }
        .section h2 { color: #30415f; border-left: 4px solid #30415f; padding-left: 15px; margin-bottom: 20px; }
        .chart-container { margin: 30px 0; padding: 20px; background: #f8f9fa; border-radius: 5px; border-left: 4px solid #30415f; }
        .chart-title { font-weight: bold; margin-bottom: 15px; color: #333; font-size: 1.2em; text-align: center; }
        .chart-description { margin-top: 15px; color: #666; font-style: italic; text-align: center; }
        .key-insights { background: #e3f2fd; padding: 20px; border-radius: 5px; margin: 20px 0; }
        .key-insights h3 { color: #30415f; margin-top: 0; }
        .footer { margin-top: 50px; text-align: center; color: #666; border-top: 1px solid #ddd; padding-top: 20px; }
        .center-chart { display: flex; justify-content: center; }
        @media print { body { background: white; } .container { box-shadow: none; } }
    </style>
</head>
<body>
<div class="container">
    <div class="header">
        <h1>{{ report_title }}</h1>
        <div class="date">Generated on: {{ current_date }}</div>
    </div>
    <div class="section">
        <h2>Executive Summary</h2>
        <p>A summary of the MSCI India Index and its valuation, earnings and composition stats.</p>
        <div class="key-insights">
            <h3>Key Insights</h3>
            <ul>
                <li>- Trading on high valuations relative to history</li>
                <li>- Has 15% earnings growth expected for 2025</li>
                <li>- Likely will benefit from a falling USD</li>
            </ul>
        </div>
    </div>
    <div class="section">
        <h2>Return & Risk Overview</h2>

        <div class="chart-container" style="overflow-x: auto;">
            <div class="chart-title">Total Return</div>
            <div style="min-width: 700px; max-width: 100%; overflow-x: auto;">{{ master_return_table | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">Returns – Since 1999</div>
            <div class="center-chart">{{ chart_99_html | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">GFC Returns</div>
            <div class="center-chart">{{ gfc_chart_html | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">Returns Since COVID</div>
            <div class="center-chart">{{ since_covid_chart_html | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">2-Year Rolling Volatility</div>
            <div class="center-chart">{{ rolling_3y_vol_chart_html | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">2-Year Rolling Return/Risk (Risk-Adjusted Return)</div>
            <div class="center-chart">{{ return_over_risk_chart_html | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">Is the Region shifting towards intangible assets?</div>
            <div class="center-chart">{{ bookvalue_win_chart_html | safe }}</div>
        </div>
    </div>

    <div class="section">
        <h2>Valuation Overview</h2>
        <div class="chart-container" style="overflow-x: auto;">
            <div class="chart-title">Traffic Light Summary</div>
            <div style="min-width: 700px; max-width: 100%; overflow-x: auto;">{{ table_traffic | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">Equity Sources of Total Return – Year to Date</div>
            <div class="center-chart">{{ decomp_html | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">Generic Valuation vs Benchmarks</div>
            <div class="center-chart">{{ fwd_pe_chart | safe }}</div>
        </div>

        <div class="chart-container">
            <div class="chart-title">Time Series Valuation</div>
            <div class="center-chart">{{ ts_val_charts | safe }}</div>
        </div>
        <div class="chart-container">
            <div class="chart-title">Cross-Sectional Analysis</div>
            <div class="center-chart">{{ xs_chart | safe }}</div>
        </div>
    </div>
    <div class="section">
        <h2>Earnings Analysis</h2>
        <div class="chart-container">
            <div class="chart-title">Earnings Momentum</div>
            <div class="center-chart">{{ earnings_imp_chart | safe }}</div>
        </div>
        <div class="chart-container">
            <div class="chart-title">Earnings Revisions</div>
            <div class="center-chart">{{ earnings_rev_chart | safe }}</div>
        </div>
    </div>
    <div class="section">
        <h2>Conclusions & Recommendations</h2>
        <ol>
            <li></li>
            <li></li>
            <li></li>
        </ol>
    </div>
    <div class="footer">
        <p>Created by Innova Asset Management and Data by Bloomberg and Innova Asset Management.</p>
    </div>
</div>
</body>
</html>
"""

# Render the HTML with all dynamic content
rendered = Template(template_html).render(
    current_date=datetime.today().strftime('%B %d, %Y'),
    report_title=name_for_report,
    table_traffic=table_traffic,
    fwd_pe_chart=fwd_pe_chart_html,
    ts_val_charts=ts_val_charts_html,
    xs_chart=xs_chart_html,
    earnings_imp_chart=earnings_imp_chart_html,
    earnings_rev_chart=earnings_rev_chart_html,
    decomp_html=decomp_html,
    master_return_table=master_return_table,
    chart_99_html=chart_99_html,
    gfc_chart_html=gfc_chart_html,
    since_covid_chart_html=since_covid_chart_html,
    rolling_3y_vol_chart_html=rolling_3y_vol_chart_html,
    bookvalue_win_chart_html=bookvalue_win_chart_html,
    return_over_risk_chart_html=return_over_risk_chart_html
)

# Save to file
with open(f'reports/index.html', "w", encoding="utf-8") as f:
    f.write(rendered)